In [11]:
# !pip install pymupdf reportlab lxml

import fitz  # PyMuPDF
from reportlab.pdfgen import canvas
from reportlab.lib.colors import CMYKColor
from lxml import etree

# -------------------------------
# Step 1. Read PDF & export first page to SVG
# -------------------------------

input_pdf = "desert.pdf"
doc = fitz.open(input_pdf)
page = doc[0]

svg_str = page.get_svg_image()
with open("step1_original.svg", "w", encoding="utf-8") as f:
    f.write(svg_str)

print("✅ Saved: step1_original.svg")


✅ Saved: step1_original.svg


In [12]:
# -------------------------------
# Step 2. Add bleed to SVG (expand viewBox)
# -------------------------------

bleed_mm = 3
pt_per_inch = 72
mm_to_inch = 1 / 25.4
bleed_points = bleed_mm * mm_to_inch * pt_per_inch

tree = etree.fromstring(svg_str.encode("utf-8"))

width = float(tree.get("width").replace("pt", ""))
height = float(tree.get("height").replace("pt", ""))
viewBox = [float(v) for v in tree.get("viewBox").split()]

# Expand viewBox for bleed
viewBox[0] -= bleed_points
viewBox[1] -= bleed_points
viewBox[2] += 2 * bleed_points
viewBox[3] += 2 * bleed_points

tree.set("width", f"{width + 2*bleed_points}pt")
tree.set("height", f"{height + 2*bleed_points}pt")
tree.set("viewBox", " ".join(str(v) for v in viewBox))

svg_bleed_str = etree.tostring(tree, encoding="unicode")
with open("step2_bleed.svg", "w", encoding="utf-8") as f:
    f.write(svg_bleed_str)

print("✅ Saved: step2_bleed.svg")

✅ Saved: step2_bleed.svg


In [13]:
# -------------------------------
# Step 3. Create PDF with bleed (without CutContour)
# -------------------------------

# Make new PDF with larger page size
new_doc = fitz.open()
new_page = new_doc.new_page(width=viewBox[2], height=viewBox[3])

# Place original PDF page into the center of bleed page
target_rect = fitz.Rect(bleed_points, bleed_points,
                        bleed_points + width, bleed_points + height)
new_page.show_pdf_page(target_rect, doc, 0)

new_doc.save("step3_bleed.pdf")
new_doc.close()
print("✅ Saved: step3_bleed.pdf")

✅ Saved: step3_bleed.pdf


In [14]:
# -------------------------------
# Step 4. Add CutContour to the bleed PDF
# -------------------------------

from reportlab.pdfgen import canvas
from PyPDF2 import PdfReader, PdfWriter

# Define spot color
spot = CMYKColor(0, 1, 0, 0, spotName="CUT")

# Get page size from bleed PDF
bleed_reader = PdfReader("step3_bleed.pdf")
page_bleed = bleed_reader.pages[0]
page_width = float(page_bleed.mediabox.width)
page_height = float(page_bleed.mediabox.height)

# Make overlay PDF with just cutline
overlay_pdf = "overlay_cutline.pdf"
c = canvas.Canvas(overlay_pdf, pagesize=(page_width, page_height))
c.setStrokeColor(spot)
c.setLineWidth(0.25)

# Draw cut line at trim box (inside bleed)
c.rect(bleed_points, bleed_points, width, height, stroke=1, fill=0)
c.save()

# Merge overlay with bleed PDF
from PyPDF2 import PdfMerger
writer = PdfWriter()
from PyPDF2 import PdfReader
import io

base = PdfReader("step3_bleed.pdf")
overlay = PdfReader(overlay_pdf)

page = base.pages[0]
page.merge_page(overlay.pages[0])
writer.add_page(page)

with open("step4_bleed_cutcontour.pdf", "wb") as f:
    writer.write(f)

print("✅ Saved: step4_bleed_cutcontour.pdf")

✅ Saved: step4_bleed_cutcontour.pdf


In [3]:
# -------------------------------
# Step 3. Create PDF with bleed (without CutContour)
# -------------------------------

# Make new PDF with larger page size
new_doc = fitz.open()
new_page = new_doc.new_page(width=viewBox[2], height=viewBox[3])

# Place original PDF page into the center of bleed page
target_rect = fitz.Rect(bleed_points, bleed_points,
                        bleed_points + width, bleed_points + height)
new_page.show_pdf_page(target_rect, doc, 0)

new_doc.save("step3_bleed.pdf")
new_doc.close()
print("✅ Saved: step3_bleed.pdf")


✅ Saved: step3_bleed.pdf


In [5]:
# -------------------------------
# Step 4. Add CutContour to the bleed PDF
# -------------------------------

from reportlab.pdfgen import canvas
from PyPDF2 import PdfReader, PdfWriter

# Define spot color
spot = CMYKColor(0, 1, 0, 0, spotName="CutContour")

# Get page size from bleed PDF
bleed_reader = PdfReader("step3_bleed.pdf")
page_bleed = bleed_reader.pages[0]
page_width = float(page_bleed.mediabox.width)
page_height = float(page_bleed.mediabox.height)

# Make overlay PDF with just cutline
overlay_pdf = "overlay_cutline.pdf"
c = canvas.Canvas(overlay_pdf, pagesize=(page_width, page_height))
c.setStrokeColor(spot)
c.setLineWidth(0.25)

# Draw cut line at trim box (inside bleed)
c.rect(bleed_points, bleed_points, width, height, stroke=1, fill=0)
c.save()

# Merge overlay with bleed PDF
from PyPDF2 import PdfMerger
writer = PdfWriter()
from PyPDF2 import PdfReader
import io

base = PdfReader("step3_bleed.pdf")
overlay = PdfReader(overlay_pdf)

page = base.pages[0]
page.merge_page(overlay.pages[0])
writer.add_page(page)

with open("step4_bleed_cutcontour.pdf", "wb") as f:
    writer.write(f)

print("✅ Saved: step4_bleed_cutcontour.pdf")


✅ Saved: step4_bleed_cutcontour.pdf


In [6]:
# -------------------------------
# Step 5. Redimension final PDF to specific size & DPI
# -------------------------------

# Target dimensions
final_width_cm = 21    # e.g. A4 width
final_height_cm = 29.7 # A4 height
dpi = 300

final_width_points = (final_width_cm / 2.54) * 72
final_height_points = (final_height_cm / 2.54) * 72

print("Target size:", final_width_points, "x", final_height_points)

# Scale factors
scale_x = final_width_points / page_width
scale_y = final_height_points / page_height
scale = min(scale_x, scale_y)

# Create resized PDF
resized_doc = fitz.open()
resized_page = resized_doc.new_page(width=final_width_points, height=final_height_points)

src_doc = fitz.open("step4_bleed_cutcontour.pdf")
src_page = src_doc[0]

# Place with scaling
resized_page.show_pdf_page(resized_page.rect, src_doc, 0, clip=None)

resized_doc.save("step5_resized.pdf")
resized_doc.close()
src_doc.close()

print("✅ Saved: step5_resized.pdf")


Target size: 595.275590551181 x 841.8897637795275
✅ Saved: step5_resized.pdf
